# Loan Default Prediction with Neural Networks

This notebook implements a neural network model for predicting loan defaults using PyTorch. The model processes both numerical and categorical features using proper preprocessing techniques.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import time

# Set device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Custom Dataset Class
We create a custom PyTorch Dataset class to handle our loan data efficiently. This class:
- Converts features to PyTorch tensors
- Handles both numpy arrays and sparse matrices
- Supports both training data (with labels) and test data (without labels)

In [5]:
class LoanDataset(Dataset):
    def __init__(self, features, labels=None):
        # Convert features to tensor depending on input type
        if isinstance(features, np.ndarray):
            self.features = torch.FloatTensor(features)
        else:  # Sparse matrix from ColumnTransformer
            self.features = torch.FloatTensor(features.toarray())

        # Convert labels to tensor if provided
        self.labels = None if labels is None else torch.LongTensor(labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.labels is None:
            return self.features[idx]
        return self.features[idx], self.labels[idx]

## Neural Network Architecture
Our model uses multiple fully connected layers with:
- ReLU activation
- Batch normalization for better training stability
- Dropout for regularization
- Binary classification output (Default vs No Default)

In [6]:
class LoanDefaultNN(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.3):
        """
        Neural Network for loan default prediction
        Args:
            input_dim: Number of input features after preprocessing
            hidden_dims: List of hidden layer dimensions
            dropout: Dropout rate
        """
        super().__init__()

        layers = []
        prev_dim = input_dim

        # Create hidden layers
        for dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.ReLU(),
                nn.BatchNorm1d(dim),
                nn.Dropout(dropout)
            ])
            prev_dim = dim

        # Output layer
        layers.append(nn.Linear(prev_dim, 2))  # Binary classification

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

## Data Preprocessing
Here we:
1. Load and preprocess the data
2. Split features into numerical and categorical
3. Apply appropriate scaling and encoding
4. Create train/validation splits and data loaders

In [7]:
# Load the data
data = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Define categorical and numerical features
categorical_features = ['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage',
                       'HasDependents', 'LoanPurpose', 'HasCoSigner']
numerical_features = ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed',
                     'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Prepare data
X = preprocessor.fit_transform(data.drop(columns=['LoanID', 'Default']))
X_test = preprocessor.transform(test_df.drop(columns=['LoanID']))
y = data['Default'].values

# Split training data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets
train_dataset = LoanDataset(X_train, y_train)
val_dataset = LoanDataset(X_val, y_val)
test_dataset = LoanDataset(X_test)  # No labels for test set

# Create dataloaders
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Training and Validation Functions
These functions handle:
- Training epochs
- Validation
- Prediction
- Progress tracking with tqdm

In [8]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc='Training'):
        features, labels = batch
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Validating'):
            features, labels = batch
            features, labels = features.to(device), labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    accuracy = (np.array(predictions) == np.array(actuals)).mean()
    return total_loss / len(dataloader), accuracy

def predict(model, dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for features in tqdm(dataloader, desc='Predicting'):
            features = features.to(device)
            outputs = model(features)
            predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())

    return predictions

## Model Training
Train the model with:
- Cross-entropy loss
- Adam optimizer
- Model checkpointing for best validation accuracy

In [9]:
# Initialize model
input_dim = X_train.shape[1]
model = LoanDefaultNN(input_dim).to(device)

# Training parameters
n_epochs = 30
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
best_val_acc = 0
for epoch in range(n_epochs):
    start_time = time.time()

    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)

    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{n_epochs} | Time: {epoch_time:.2f}s")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

Validating: 100%|██████████| 80/80 [00:00<00:00, 259.06it/s]


Epoch 1/30 | Time: 3.03s
Train Loss: 0.4533 | Val Loss: 0.3204 | Val Acc: 0.8843


Validating: 100%|██████████| 80/80 [00:00<00:00, 290.23it/s]


Epoch 2/30 | Time: 2.18s
Train Loss: 0.3307 | Val Loss: 0.3185 | Val Acc: 0.8841


Validating: 100%|██████████| 80/80 [00:00<00:00, 282.59it/s]


...

Epoch 28/30 | Time: 2.28s
Train Loss: 0.3085 | Val Loss: 0.3189 | Val Acc: 0.8846


Validating: 100%|██████████| 80/80 [00:00<00:00, 271.49it/s]


Epoch 29/30 | Time: 2.50s
Train Loss: 0.3076 | Val Loss: 0.3181 | Val Acc: 0.8849


Validating: 100%|██████████| 80/80 [00:00<00:00, 289.28it/s]

Epoch 30/30 | Time: 2.45s
Train Loss: 0.3074 | Val Loss: 0.3184 | Val Acc: 0.8850


## Generate Predictions
Generate predictions on the test set using the best model and create submission file.

In [10]:
# Load best model
model.load_state_dict(torch.load('best_model.pth'))
predictions = predict(model, test_loader)

# Create submission file
submission = pd.DataFrame({
    'LoanID': test_df['LoanID'],
    'Default': predictions
})
submission.to_csv('neural_network_submission.csv', index=False)

<ipython-input-10-2f1803d3d410>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
Predicting: 100%|██████████| 100/100 [00